In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd
import math

In [5]:
human_slides = ["10835_19 humane BAL Turnbull Blau.svs",
                "10120_19 humane BAL Berliner Blau.svs",
                "10052_19 humane BAL Berliner Blau-001.svs",
                "10080_19 humane BAL Berliner Blau-001.svs",
                "2707_20 TB BAL Human neu.svs",
                "10835_19 humane BAL Berliner Blau.svs",
                "10227_19 humane BAL Berliner Blau.svs",
                "11480_19 humane BAL Berliner Blau.svs",
                "11323_19 humane BAL Berliner Blau.svs",
                "2702_20 BB Human BAL-001.svs",
                "10672_19 humane BAL Berliner Blau.svs",
                "10672_19 humane BAL Turnbull Blau.svs"]

In [8]:
fold_1 = pickle.load(open(str("inference_results_Equine-Fold-1.p"),'rb'))
fold_2 = pickle.load(open(str("inference_results_Equine-Fold-2.p"),'rb'))
fold_3 = pickle.load(open(str("inference_results_Equine-Fold-3.p"),'rb'))

In [20]:
folds = {}
nms_thresh = 40
thresh = 0.35


for file_name in tqdm(human_slides):
    
    boxes1 = np.array(fold_1[file_name])
    boxes2 = np.array(fold_2[file_name])
    boxes3 = np.array(fold_3[file_name])
    
    boxes = np.concatenate([boxes1, boxes2, boxes3])
    init_shape = boxes.shape
    
    scales = ((boxes[:, 2] - boxes[:, 0]) / (boxes[:, 3] - boxes[:, 1]))
    boxes = boxes[(scales < 1.1) & (scales > 0.9)]
    
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    final_shape = annos.shape    
    
    #print("Input: {}  Output: {}".format(init_shape, final_shape))
    
    folds[file_name] = annos

100%|██████████| 12/12 [00:50<00:00,  2.21s/it]


In [22]:
pickle.dump(folds, open(str("InferenceAnnotationsHumaneV2.p"), "wb"))